# Helpful Functions



In [5]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

#### Imports and Utility Functions

The following imports are needed to create the end-to-end memory network. Neither Keras nor TensorFlow directly support End-to-End Memory Networks (yet), so it is necessary to create them using existing tools.  Several functions are needed defined here to read the bAbI dataset that we are using to train.

In [6]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from functools import reduce
import pickle
import tarfile
import numpy as np
import re
import os
import time


def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)', sent) if x.strip()]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data):
    inputs, queries, answers = [], [], []
    for story, query, answer in data:
        inputs.append([word_idx[w] for w in story])
        queries.append([word_idx[w] for w in query])
        answers.append(word_idx[answer])
    return (pad_sequences(inputs, maxlen=story_maxlen),
            pad_sequences(queries, maxlen=query_maxlen),
            np.array(answers))


C:\Users\DELL\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Getting the Data

The data is downloaded from the Internet, if needed.

As you can see (below), this dataset contains stories and questions about those stories.  The computer is not learning these specific stories below, but rather how to read a story and answer a question about that story.  Consider the first story "Mary moved to the bathroom. John went to the hallway." the computer is not learning that Mary is in the bathroom or John is in the hallway, this changes per story.  Rather, the computer is learning to parse the story and extract information about individual people and their locations.

The computer is learning to read, at least in a limited sense.

In [7]:
# try:
#     path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
# except:
#     print('Error downloading dataset, please download it manually:\n'
#           '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
#           '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
#     raise
path = 'J:SONGS/LAP DOWNLOAD/GEETA/Compressed/tasks_1-20_v1-2.tar.gz'
tar = tarfile.open(path)

challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))



Extracting stories for the challenge: single_supporting_fact_10k


In [8]:
# See what the data looks like

for i in range(5):
    print("Story: {}".format(' '.join(train_stories[i][0])))
    print("Query: {}".format(' '.join(train_stories[i][1])))
    print("Answer: {}".format(train_stories[i][2]))
    print("--- --------")

Story: Mary moved to the bathroom . John went to the hallway .
Query: Where is Mary ?
Answer: bathroom
--- --------
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden .
Query: Where is Daniel ?
Answer: hallway
--- --------
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom .
Query: Where is Daniel ?
Answer: hallway
--- --------
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom . Mary moved to the hallway . Daniel travelled to the office .
Query: Where is Daniel ?
Answer: office
--- --------
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra 

#### Building the Vocabulary

This type of neural network can only deal with a set vocabulary.  The words are indexed and each becomes a number.  Words not in the training vocabulary will not be recognized.

In [9]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')


for s in list(enumerate(vocab)):
    print(s)

-
Vocab size: 22 unique words
Story max length: 68 words
Query max length: 4 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom')
-
(0, '.')
(1, '?')
(2, 'Daniel')
(3, 'John')
(4, 'Mary')
(5, 'Sandra')
(6, 'Where')
(7, 'back')
(8, 'bathroom')
(9, 'bedroom')
(10, 'garden')
(11, 'hallway')
(12, 'is')
(13, 'journeyed')
(14, 'kitchen')
(15, 'moved')
(16, 'office')
(17, 'the')
(18, 'to')
(19, 'travelled')
(20, 'went')


#### Building the Training and Test Data

The training data that is actually sent to the neural network is the vectorized representation of the sentences.  Each word is replaced by its vocab number.  Additionally there are two parts to the input (x) data: story and query.  The answer (x) is a always a single vocab word number.  This is a classification network.  Any of the vocab words could potentially be the answer.  Stories can be at most 68 words and questions at most 4.  Both of these limits are automatically determined from the training data.

In [10]:
print('Vectorizing the word sequences...')
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')


Vectorizing the word sequences...
-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000,)
answers_test shape: (1000,)
-


In [11]:
# See individual training element.

print("Story (x): {}".format(inputs_train[0]))
print("Question (x): {}".format(queries_train[0]))
print("Answer: {}".format(answers_train[0]))

Story (x): [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  5 16 19 18  9  1  4 21 19 18 12  1]
Question (x): [ 7 13  5  2]
Answer: 9


#### Compile the Neural Network

In [12]:
print('Compiling...')

# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print("Done.")


Compiling...
Done.


#### Train the Neural Network

It will take some time (probably up to 1/2 hour) to train this network on a CPU.  The network is saved.  If you've previously saved the neural network, you can skip this step and load the neural network in the next step.

In [19]:
start_time = time.time()
# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))

# save
save_path = "C:/Users/DELL"
# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(save_path,"chatbot.h5"))
# save the vocab too, indexes must be the same
pickle.dump( vocab, open( os.path.join(save_path,"vocab.pkl"), "wb" ) )

elapsed_time = time.time() - start_time
#print("Elapsed time: {}".format(hms_string(elapsed_time)))

Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 5s 484us/step - loss: 0.3462 - acc: 0.8704 - val_loss: 0.7209 - val_acc: 0.7700
Epoch 2/120
10000/10000 [==============================] - 5s 452us/step - loss: 0.3380 - acc: 0.8747 - val_loss: 0.7202 - val_acc: 0.7740
Epoch 3/120
10000/10000 [==============================] - 4s 424us/step - loss: 0.3345 - acc: 0.8743 - val_loss: 0.7296 - val_acc: 0.7750
Epoch 4/120
10000/10000 [==============================] - 4s 430us/step - loss: 0.3315 - acc: 0.8763 - val_loss: 0.7338 - val_acc: 0.7800
Epoch 5/120
10000/10000 [==============================] - 4s 425us/step - loss: 0.3301 - acc: 0.8748 - val_loss: 0.7307 - val_acc: 0.7750
Epoch 6/120
10000/10000 [==============================] - 4s 426us/step - loss: 0.3294 - acc: 0.8776 - val_loss: 0.7137 - val_acc: 0.7800
Epoch 7/120
10000/10000 [==============================] - 4s 430us/step - loss: 0.3261 - acc: 0.8811 - val_loss: 0.7

10000/10000 [==============================] - 4s 414us/step - loss: 0.2018 - acc: 0.9258 - val_loss: 0.7882 - val_acc: 0.8050
Epoch 60/120
10000/10000 [==============================] - 4s 421us/step - loss: 0.2049 - acc: 0.9230 - val_loss: 0.7675 - val_acc: 0.7910
Epoch 61/120
10000/10000 [==============================] - 4s 426us/step - loss: 0.2086 - acc: 0.9247 - val_loss: 0.7808 - val_acc: 0.7930
Epoch 62/120
10000/10000 [==============================] - 4s 443us/step - loss: 0.2003 - acc: 0.9273 - val_loss: 0.7884 - val_acc: 0.8050
Epoch 63/120
10000/10000 [==============================] - 4s 415us/step - loss: 0.1991 - acc: 0.9269 - val_loss: 0.8116 - val_acc: 0.7930
Epoch 64/120
10000/10000 [==============================] - 4s 384us/step - loss: 0.1965 - acc: 0.9293 - val_loss: 0.7967 - val_acc: 0.7960
Epoch 65/120
10000/10000 [==============================] - 4s 373us/step - loss: 0.1948 - acc: 0.9266 - val_loss: 0.7931 - val_acc: 0.7930
Epoch 66/120
10000/10000 [=======

10000/10000 [==============================] - 4s 446us/step - loss: 0.1543 - acc: 0.9453 - val_loss: 0.9131 - val_acc: 0.7940
Epoch 118/120
10000/10000 [==============================] - 4s 387us/step - loss: 0.1511 - acc: 0.9478 - val_loss: 0.8945 - val_acc: 0.8000
Epoch 119/120
10000/10000 [==============================] - 4s 390us/step - loss: 0.1514 - acc: 0.9462 - val_loss: 0.9367 - val_acc: 0.7990
Epoch 120/120
10000/10000 [==============================] - 4s 389us/step - loss: 0.1555 - acc: 0.9457 - val_loss: 0.8711 - val_acc: 0.7950


In [20]:
# Load the model, if it exists, load vocab too
save_path = "C:/Users/DELL"
model = load_model(os.path.join(save_path,"chatbot.h5"))
vocab = pickle.load( open( os.path.join(save_path,"vocab.pkl"), "rb" ) )

#### Evaluate Accuracy

We evaluate the accuracy, using the same technique as previous classification networks.

In [21]:
pred = model.predict([inputs_test, queries_test])
# See what the predictions look like, they are just probabilities of each class.
print(pred)

[[4.08919928e-14 3.90611107e-14 3.49505005e-14 ... 3.53285076e-14
  3.76819615e-14 3.07150240e-14]
 [8.19659705e-12 7.73657267e-12 7.54286824e-12 ... 7.67844555e-12
  8.27746031e-12 7.06611072e-12]
 [1.15509841e-14 1.03843774e-14 1.17141031e-14 ... 1.03380143e-14
  1.09894470e-14 1.02600686e-14]
 ...
 [1.11242006e-14 9.21831123e-15 9.48332582e-15 ... 9.60419403e-15
  1.00340184e-14 8.74728638e-15]
 [4.63206811e-14 4.00618361e-14 4.19365775e-14 ... 3.88197809e-14
  4.56455349e-14 3.40618274e-14]
 [6.62211674e-11 6.48707615e-11 6.38643027e-11 ... 6.47958215e-11
  6.55933849e-11 5.53883571e-11]]


In [24]:
# Use argmax to turn those into actual predictions.  The class (word) with the highest
# probability is the answer.

pred = np.argmax(pred,axis=1)
print(pred)

[12  9 15 12 15 12 11 12 17 15 15 11 17 10 10 15 11 15  9  9 12 17 12 12
 10 10 10 15 17 11 12 17 15 15  9 17  9 11 10 10  9 11 11 10 11 15 12  9
 17 10 10 17 11 17 17 10 15 12 12 12 12 17  9 11 10 15 15  9 17  9 11 12
 11 10  9 15 11 15 15 17 11  9 10 10 17 11 11 12 15 15  9  9 12  9 10 15
 15 15 10 15 10  9 17 11 11 12 10  9 17 10  9 12 15 12 15 11 11 15  9 17
 10  9  9 12 10 17 11 11 11  9 11 11  9 11 11  9 17 15  9  9 17 12 17 17
 10 15 11 15 15 11 11 11  9 17 10  9 11 10 17 12 10 11 10 10 11 11 17 11
 15 10 17 12 11  9 10 17 15 10 17 11 12 17 12 12 10  9 17 11 15 15 12 15
 11 12 10 11 11 11  9 10 12 17 12 10 15  9 15 12  9 11  9 10 12 10  9 10
  9 12 12 12  9 12 15 15 10 17 12 11  9 17 17 11 12 17 17 10 10  9 10 17
 12 15 11 11 15 15 11 15 17 11 10 17  9 11 10 10 17 10 15 11 11 17 12 17
 17  9 11 10 12 10 15 17 12 15 15 17  9 12 17  9  9 11 17 17 11 17  9 11
 11  9 15  9 10 15 15 11 11 10 12  9  9 15 12 17 17 11  9 17  9 15 11 10
 17 10 15 17 10 17 11 11 11 15 15 11 11  9 10 11 11

In [25]:
score = metrics.accuracy_score(answers_test, pred)
print("Final accuracy: {}".format(score))

Final accuracy: 0.795


#### Adhoc Query

You might want to create your own stories and questions.  

In [26]:
print("Remember, I only know these words: {}".format(vocab))
print()
story = "Daniel went to the hallway. Mary went to the bathroom.  Daniel went to the bedroom."
query = "Where is Daniel?"

adhoc_stories = (tokenize(story), tokenize(query), '?')

adhoc_train, adhoc_query, adhoc_answer = vectorize_stories([adhoc_stories])

pred = model.predict([adhoc_train, adhoc_query])
print(pred[0])
pred = np.argmax(pred,axis=1)
print("Answer: {}({})".format(vocab[pred[0]-1],pred))


Remember, I only know these words: ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']

[3.0556726e-15 2.8733215e-15 2.8921480e-15 3.0907250e-15 2.8470920e-15
 2.9356089e-15 3.0561622e-15 3.1034137e-15 2.8667525e-15 1.4291936e-07
 9.9999917e-01 6.9781362e-07 4.5414933e-10 2.8447254e-15 3.1162732e-15
 3.9044912e-13 3.5632969e-15 1.5377085e-11 3.0660412e-15 2.8612790e-15
 2.9970965e-15 2.6466341e-15]
Answer: bedroom([10])
